# 🧙‍♂️ Gandalf RAG Chatbot using Hugging Face Hub

This notebook runs a Retrieval-Augmented Generation (RAG) chatbot using a vectorstore index + a remote LLM from Hugging Face Inference API.

In [18]:
'''
# 📁 Step 1: Load and Split Your PDF

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
# Make sure your PDF is in the same folder or provide a full path
pdf_path = Path("Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf")
loader = PyPDFLoader(str(pdf_path))
docs = loader.load()
# Split into chunks for embedding
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(docs)
print(f"✅ Loaded and split {len(docs)} chunks from the PDF")
    
# 🧠 Step 2: Generate Embeddings and Save FAISS Index"),
    
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
# Use MiniLM (small, fast, good enough for most use cases)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("gandalf_index")
print("✅ Vectorstore saved as 'gandalf_index'")'
'''

'\n# 📁 Step 1: Load and Split Your PDF\n\nfrom langchain.document_loaders import PyPDFLoader\nfrom langchain.text_splitter import RecursiveCharacterTextSplitter\nfrom pathlib import Path\n# Make sure your PDF is in the same folder or provide a full path\npdf_path = Path("Tolkien-J.-The-lord-of-the-rings-HarperCollins-ebooks-2010.pdf")\nloader = PyPDFLoader(str(pdf_path))\ndocs = loader.load()\n# Split into chunks for embedding\nsplitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)\ndocs = splitter.split_documents(docs)\nprint(f"✅ Loaded and split {len(docs)} chunks from the PDF")\n\n# 🧠 Step 2: Generate Embeddings and Save FAISS Index"),\n\nfrom langchain.embeddings import HuggingFaceEmbeddings\nfrom langchain.vectorstores import FAISS\n# Use MiniLM (small, fast, good enough for most use cases)\nembeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")\nvectorstore = FAISS.from_documents(docs, embeddings)\nvectorstore.save_local("gan

In [19]:
# 📦 Install required packages (if running in Colab or HF Spaces)
# !pip install langchain langchain-community sentence-transformers faiss-cpu python-dotenv


## 🔐 Setup Environment

In [20]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not hf_token:
    raise ValueError("❌ Missing Hugging Face API token. Add HUGGINGFACEHUB_API_TOKEN to your .env file.")


## 🔎 Load Embeddings + Vectorstore

In [36]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local("gandalf_index", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

## 🤖 Load a Hugging Face Model for Inference

In [50]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 🧙 Custom prompt template
custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Use the following context to answer the question. 
If you don't know the answer, just say you don't know — do not make up an answer.

Context:
{context}

Question:
{question}

Answer:
"""
)

# 🛠️ RAG setup with custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}
)


In [51]:
llm = HuggingFaceEndpoint(
    repo_id="tiiuae/falcon-7b-instruct",
    huggingfacehub_api_token=hf_token,
    temperature=0.7,
    max_new_tokens=250
)


In [52]:
question = "What happened in the mines of Moria?"
result = qa_chain.invoke({"query": question})

print("🧙 Gandalf says:\n", result['result'])


🧙 Gandalf says:
 The Dark Lord Sauron had long been aware of the existence of the mine, and 
had taken control of it to store his immense wealth. The mines were guarded 
by a powerful army of Orcs, and the area was heavily fortified. Gandalf 
told the dwarves that the Dark Lord had taken the mines from their ancestors 
and used them as his treasure trove. The mines were also the source of 
the metal that was in high demand by the dwarves for their intricate 
jewellery.

The mine was destroyed by the dwarves, and the Dark Lord's power over 
the region was weakened. However, he still retained his power over the 
mountain and its surrounding areas.


In [42]:
docs = retriever.get_relevant_documents(question)
print("\n--- Retrieved Documents ---\n")
for doc in docs:
    print(doc.page_content[:500], "\n---")



--- Retrieved Documents ---

was then again growing in the world, though the Shadow in the Forest that 
looked towards Moria was not yet known for what it was. All evil things 
were stirring. The Dwarves delved deep at that time, seeking beneath Baraz-
inbar for mithril, the metal beyond price that was becoming yearly ever harder 
The Hobbit,p. 50. 1 
---
them eastwards, and one on either side. Then the light went out. 
‘That is all that I shall venture on for the present,’ said Gandalf. 
‘There used to be great windows on the mountain-side, and shafts 
leading out to the light in the upper reaches of the Mines. I think 
we have reached them now, but it is night outside again, and we 
cannot tell until morning. If I am right, tomorrow we may actually 
see the morning peeping in. But in the meanwhile we had better go 
---
plundered, and they became a wandering people. Moria for long remained 
secure, but its numbers dwindled until many of its vast mansions became 
dark and empty. The w

C:\Users\JonMa\AppData\Local\Temp\ipykernel_12028\3599709441.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


## 🔁 Ask Questions (RAG)

In [46]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

question = "What happened in the mines of Moria?"
result = qa_chain.invoke({"query": question})

print("🧙 Gandalf says:\n", result['result'])

🧙 Gandalf says:
 
Go to the The Hobbit, Chapter 1, 'There and Back Again' page
for more information and analysis.
Take a Study Break!
